In [3]:
import pandas as pd
import geopandas as gpd

# 1. Read the CSV with lichtpunten
punten = pd.read_csv("data/LICHTPUNTEN.csv", sep=";")

# Turn the points into a GeoDataFrame  WGS84 is EPSG 4326
gdf_punten = gpd.GeoDataFrame(
    punten,
    geometry=gpd.points_from_xy(punten["LNG"], punten["LAT"]),
    crs="EPSG:4326"
)

# 2. Read the wijken from the geopackage
wijken = gpd.read_file("data/wijkenbuurten_2023_v3.gpkg", layer="wijken")

# Make sure CRS is set correctly  CBS wijkenbuurten is in RD New  EPSG 28992
if wijken.crs is None:
    wijken = wijken.set_crs(epsg=28992)

# Reproject wijken to WGS84 so both layers match
wijken_4326 = wijken.to_crs("EPSG:4326")

# Keep only the fields you care about
wijken_4326 = wijken_4326[["wijkcode", "wijknaam", "gemeentenaam", "geometry"]]

# 3. Spatial join  assign each point to the wijk polygon it falls within
punten_met_wijk = gpd.sjoin(
    gdf_punten,
    wijken_4326,
    how="left",
    predicate="within"
)

# 4. Save back to CSV  without the geometry column
kolommen = [col for col in punten_met_wijk.columns if col != "geometry"]
punten_met_wijk[kolommen].to_csv("LICHTPUNTEN_met_wijk.csv", sep=";", index=False)


In [4]:
punten_met_wijk

,OBJECTNUMMER,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,geometry,index_right,wijkcode,wijknaam,gemeentenaam
0,1,POINT(4.897456 52.3767509),POINT(52.3767509 4.897456),4.897456,52.376751,POINT (4.89746 52.37675),976,WK0363AD,Burgwallen-Nieuwe Zijde,Amsterdam
1,2,POINT(4.794352 52.399647),POINT(52.399647 4.794352),4.794352,52.399647,POINT (4.79435 52.39965),983,WK0363BA,Havens-West,Amsterdam
2,3,POINT(4.942965 52.364033),POINT(52.364033 4.942965),4.942965,52.364033,POINT (4.94296 52.36403),1042,WK0363MG,Indische Buurt-Oost,Amsterdam
3,4,POINT(4.931492 52.365037),POINT(52.365037 4.931492),4.931492,52.365037,POINT (4.93149 52.36504),1041,WK0363MF,Indische Buurt-West,Amsterdam
4,5,POINT(4.8698671 52.404552),POINT(52.404552 4.8698671),4.869867,52.404552,POINT (4.86987 52.40455),984,WK0363BB,Coenhaven/Minervahaven,Amsterdam
...,...,...,...,...,...,...,...,...,...,...
131481,131482,POINT(4.9216231 52.394225),POINT(52.394225 4.9216231),4.921623,52.394225,POINT (4.92162 52.39422),1061,WK0363NL,IJplein/Vogelbuurt,Amsterdam
131482,131483,POINT(4.92156 52.394008),POINT(52.394008 4.92156),4.921560,52.394008,POINT (4.92156 52.39401),1061,WK0363NL,IJplein/Vogelbuurt,Amsterdam
131483,131484,POINT(4.9213379 52.391747),POINT(52.391747 4.9213379),4.921338,52.391747,POINT (4.92134 52.39175),1061,WK0363NL,IJplein/Vogelbuurt,Amsterdam
131484,131485,POINT(4.9233119 52.394445),POINT(52.394445 4.9233119),4.923312,52.394445,POINT (4.92331 52.39444),1061,WK0363NL,IJplein/Vogelbuurt,Amsterdam


In [1]:
print('HAllo')

HAllo
